In [ ]:
import copy
import json
import pathlib

from distutils import dir_util
from dask.distributed import Client, LocalCluster
from urllib.request import urlretrieve

from laserfarm import Retiler, DataProcessing, GeotiffWriter, Classification
from laserfarm import MacroPipeline

# Macro-ecology LiDAR point-cloud processing pipeline 

## 0. Data Retrieval  and Cluster Setup

Files produced by the pipeline will be saved in the `tmp_folder` directory. 

In [ ]:
tmp_folder = pathlib.Path('/var/tmp')

We start by checking whether the test data set is available locally, we otherwise retrieve it from the AHN3 repository.

In [ ]:
testdata_files = ['C_41CZ2.LAZ']

file_paths = [tmp_folder/f for f in testdata_files]

for file_path in file_paths:
    if not file_path.is_file():
        url = 'https://geodata.nationaalgeoregister.nl/ahn3/extract/ahn3_laz'
        url = '/'.join([url, file_path.name])
        urlretrieve(url, file_path)

We then setup the cluster that we will use for the computation using `dask`. For this example, the cluster consists of 2 processes (workers). Note: it is important that single-threaded workers are employed for the tasks that require `laserchicken`!  

In [ ]:
cluster = LocalCluster(processes=True, 
                       n_workers=2, 
                       threads_per_worker=1, 
                       local_directory=tmp_folder/'dask-worker-space')
cluster

## 1. Retiling

The first step in the pipeline is to retile the retrieved point-cloud files to a regular grid, splitting the original data into smaller chuncks that are easier to handle for data processing. The boundaries of the grid and the number of tiles along each axis are set to:

In [ ]:
grid = {
    'min_x': -113107.8100,
    'max_x': 398892.1900,
    'min_y': 214783.8700,
    'max_y': 726783.87,
    'n_tiles_side': 256
}

The retiling of multiple input files consists of independent tasks, which are thus efficiently parallelized. The input controlling all the steps of the retiling is organized in a dictionary.

In [ ]:
# set path where output will be written 
retiling_out_path = tmp_folder/'retiled'

retiling_input = {
    'setup_local_fs': {
        'input_folder': tmp_folder.as_posix(),
        'output_folder': retiling_out_path.as_posix()
    },
    'set_grid': grid,
    'split_and_redistribute': {},
    'validate': {}
}

In [ ]:
retiling_macro = MacroPipeline()

for file_path in file_paths:
    retiler = Retiler(input_file=file_path.name, label=file_path.stem)
    retiler.config(retiling_input)
    retiling_macro.add_task(retiler)

retiling_macro.setup_cluster(cluster=cluster)

# run!
retiling_macro.run()
retiling_macro.print_outcome()

## 2. Feature Extraction

Once the files are splitted into tiles of a manageable size, we proceed to the feature extraction stage, which is performed using `laserchicken`. We choose the following two example features:

In [ ]:
feature_names = ['mean_normalized_height', 'std_normalized_height']

The base input dictionary for this step looks like:

In [ ]:
# set path where output will be written 
dp_out_path = tmp_folder/'targets'

dp_input = {
    'setup_local_fs': {
        'input_folder': retiling_out_path.as_posix(),
        'output_folder': dp_out_path.as_posix()
    },
    'load': {},
    'normalize': {
        'cell_size': 1
    },
    'generate_targets': {
        'tile_mesh_size' : 10.0,
        'validate' : True,
        'validate_precision': 1.e-3,
        **grid
    },
    'extract_features': {
        'feature_names': feature_names,
        'volume_type': 'cell',
        'volume_size': 10
    },
    'export_targets': {},
    'clear_cache': {}
}

Note: `laserchicken` caches the KDTree computed for the point cloud. In order to free up the memory of the `dask` workers at the end of each tile's feature extraction, we need to clear the cache (see `clear_cache` in the input dictionary above).

The tiles to which the original input file has been retiled are listed in a record file located in the retiling output directory:

In [ ]:
tiles = []
for file_path in file_paths:
    record_file = '_'.join([file_path.stem, 'retile_record.js'])
    with pathlib.Path(retiling_out_path/record_file).open() as f:
        record = json.load(f)
    assert record['validated']
    tiles += [pathlib.Path(retiling_out_path/tile)
              for tile in record['redistributed_to']]
print([t.as_posix() for t in tiles])

Each tile can be processed independently, so that again one can run the tasks in a parallel fashion.

In [ ]:
dp_macro = MacroPipeline()

for tile in tiles:
    # parse tile index from the directory name
    tile_index = [int(n) for n in tile.name.split('_')[1:]]
    dp = DataProcessing(input=tile.name, label=tile.name, tile_index=tile_index)
    dp.config(dp_input)
    dp_macro.add_task(dp)
    
dp_macro.setup_cluster(cluster=cluster)

# run!
dp_macro.run()
dp_macro.print_outcome()

## 3. Classification of target points
We can classify the target points according to their groud type, based on given cadaster data. 
To mark the types of the points in the point cloud, we can add a new column `ground_type` to the target point cloud. We can use the class code of TOP10NL as the identifier. 

0. Unclassified
1. Gebouw
2. Inrichtingselement
3. Terrein (Polygon)
4. Spoorbaandeel
5. Waterdeel
6. GeografischGebied (Point)
7. FunctioneelGebied
8. Plaats
9. RegistratiefGebied
10. Hoogte
11. Relief (Line String)
12. Wegdeel

Here we present an example where we classify the points that fall on waterbodies with the given shp files of waterbody polygons. We will classify the target points according to the shape files provided in `testdata/shp`:

In [ ]:
shp_path = pathlib.Path('./testdata/shp/')

The pipeline will automatically find out the relavant shp file. We will add a new column `ground_type`, and mark all points which fall in the waterbody polygons with `5`, which is the `waterdeel` identifier.

We set up the input for the classification pipeline as follow:

In [ ]:
# set path where output will be written 
cl_out_path = tmp_folder/'classified_target_point'

classification_input = {
    'setup_local_fs': {
        'input_folder': dp_out_path.as_posix(),
        'output_folder': cl_out_path.as_posix()
    },
    'locate_shp': {'shp_dir': shp_path.absolute().as_posix()},
    'classification': {'ground_type': 5},
    'export_point_cloud': {'overwrite':True}
}

Then we excute the pipeline:

In [ ]:
cl_macro = MacroPipeline()

for tile in tiles:
    tile_path = (dp_out_path/tile.name).with_suffix('.ply') 
    cl = Classification(input_file=tile_path.as_posix(), label=tile.name)
    cl.config(classification_input)
    cl_macro.add_task(cl)
    
cl_macro.setup_cluster(cluster=cluster)

# run!
cl_macro.run()
cl_macro.print_outcome()

## 4. GeoTIFF Export

The last step of the pipeline is the transformation of the features extracted and added gound type from the point-cloud data and 'rasterized' in the target grid to a GeoTIFF file. In this case, the construction of the geotiffs (one per feature) can be performed in parallel: 

In [ ]:
# set path where output will be written 
gw_out_path = tmp_folder/'geotiffs'

gw_input = {
    'setup_local_fs': {'input_folder': cl_out_path.as_posix(),
                       'output_folder': gw_out_path.as_posix()},
    'parse_point_cloud': {},
    'data_split': [1, 1],
    'create_subregion_geotiffs': {'output_handle': 'geotiff'}
}

In [ ]:
geotiff_macro = MacroPipeline()
feature_names.append('ground_type')

for feature_name in feature_names:
    gw = GeotiffWriter(bands=feature_name, label=feature_name)
    gw.config(gw_input)
    geotiff_macro.add_task(gw)

geotiff_macro.setup_cluster(cluster=cluster)

# run!
geotiff_macro.run()
geotiff_macro.print_outcome()

Finally, we stop the client and the scheduler of the cluster.

In [ ]:
cluster.close()